In [1]:
import numpy as np
import csv
import random

FILE_PATH = "miRBase_set.csv" 
FILE_PATH_PUTATIVE = "putative_mirtrons_set.csv"

# read csv file and generate the train and test data set
def generate_data(file1_path,file2_path):
    csv_reader=csv.reader(open(file1_path, encoding='utf-8'))
    csv_reader_putative=csv.reader(open(file2_path, encoding='utf-8'))

    all_data_set = []
    tansformed_all_data_set = []
    y_all_data_set = []
    
    training_x_data=[]
    training_y_data=[]
    test_x_data=[]
    test_y_data=[]

#     read the data into a list(name,sequence,class)
    for row in csv_reader:
        all_data_set.append([row[0],row[1],row[2]])
    for row in csv_reader_putative:
        all_data_set.append([row[0],row[1],row[2]])    
#     shuffle the data set randomly    
    random.seed(2)
    random.shuffle(all_data_set)

#     get the maxmium length of the seqence
    max_seq_len = 0
    for item in all_data_set:
        if len(item[1])>max_seq_len:
            max_seq_len = len(item[1])
#     print(max_seq_len)
    
#   序列用N填充到长度都为max_seq_len
    for item in all_data_set:
        item[1] += "N" *(max_seq_len-len(item[1]))

    
#     tranformation of data set:word embedding,one_hot vector
    x_cast = {"A":[[0],[1],[0],[0],[0]],"U":[[0],[0],[1],[0],[0]],\
              "T":[[0],[0],[1],[0],[0]],"G":[[0],[0],[0],[1],[0]],\
              "C":[[0],[0],[0],[0],[1]],"N":[[0],[0],[0],[0],[0]]}
    y_cast = {"TRUE": [1,0],"FALSE":[0,1]} #TRUE:Mirtrons  FALSE:canonical microRN
    
    for item in all_data_set:
        data = []
        for char in item[1]:
            data.append(x_cast[char])
        tansformed_all_data_set.append([item[0],data,y_cast[item[2]]])
        
    
    i = j = 0
    for item in tansformed_all_data_set:
        if item[2]==[1,0]:
            i = i + 1
            if i <= 201:
                test_x_data.append(item[1])
                test_y_data.append(item[2])
            else:
                training_x_data.append(item[1])
                training_y_data.append(item[2])
        else:
            j = j + 1
            if  j<= 200:
                test_x_data.append(item[1])
                test_y_data.append(item[2])
            else:
                training_x_data.append(item[1])
                training_y_data.append(item[2])

    return training_x_data,training_y_data,test_x_data,test_y_data
    
training_x_data,training_y_data,test_x_data,test_y_data =\
generate_data(FILE_PATH,FILE_PATH_PUTATIVE)

training_x_data = np.array(training_x_data)
print(training_x_data.shape)

test_x_data = np.array(test_x_data)
print(test_x_data.shape)



(723, 164, 5, 1)
(401, 164, 5, 1)


In [2]:
# convolution with one filter of size 3
# max-pooling over the entire feature map

import tensorflow as tf
import math

#hyperparameters
LR = 0.001       #learning rate
TRAINING_ITER = 3000   #iteration times
BATCH_SIZE = 180        #batch size of input

SEQUENCE_LENGTH = 164   #sequence length of input
EMBEDDING_SIZE = 5      #char embedding size(sequence width of input)

CONV_SIZE = 6    #first filter size
CONV_DEEP = 128   #number of first filter(convolution deepth)

STRIDES = [1,1,1,1]  #the strid in each of four dimensions during convolution
KSIZE = [1,164,1,1]    #pooling window size

FC_SIZE = 128     #nodes of full-connection layer
NUM_CLASSES = 2   # classification number

DROPOUT_KEEP_PROB = 0.5   #keep probability of dropout
dataset_size = len(training_x_data)  #number of training dataset

tf.reset_default_graph()
#define placeholder
input_xs = tf.placeholder(tf.float32,[None,SEQUENCE_LENGTH,EMBEDDING_SIZE,1])
input_ys = tf.placeholder(tf.float32,[None, NUM_CLASSES])
keep_prob = tf.placeholder(tf.float32)

# function of initialize weights
def get_weights_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.01)
    weights = tf.Variable(initial,name = "weights")
    return weights

# function of convolution and pooling
def conv_and_pooling(input_tensor,filter_size,filter_width,depth,conv_deep,layer_name):
    with tf.name_scope(layer_name):
        conv_weights = get_weights_variable\
                    ([filter_size,filter_width,depth,conv_deep])
        conv_bias = tf.Variable(tf.constant(0.1,shape=[conv_deep]),name = "bias")   
        conv = tf.nn.conv2d(input_tensor,conv_weights,strides = STRIDES,padding='SAME')
        relu = tf.nn.relu(tf.nn.bias_add(conv,conv_bias))
        pool = tf.nn.max_pool(relu,ksize=KSIZE,strides=STRIDES,padding='VALID')
#     tensorboard visualization
        tf.summary.histogram("conv_weights",conv_weights)
        tf.summary.histogram("conv_bias",conv_bias)
        return pool

# design the CNN structure
def cnn_inference():
#     layer of convolution and max-pooling
    pool = conv_and_pooling(input_xs,CONV_SIZE,EMBEDDING_SIZE,1,CONV_DEEP,"filter1") 
    
#     concatenant
#     pool = tf.concat([pool],1)
#     tf.summary.scalar("concate_the_filters",pool)
    # transformation of four-dimension into two-dimension tensor,the first dimension 
    # is the batch size and the second dimension is from each sequence
#     pool_shape = pool3.get_shape().as_list()
#     nodes = pool_shape[1]*pool_shape[2]*pool_shape[3]
#     reshaped = tf.reshape(pool3,[-1,nodes])

#     pool = conv_and_pooling(input_xs,CONV_SIZE,EMBEDDING_SIZE,1,CONV_DEEP)
    
    pool_shape = pool.get_shape().as_list()
    nodes = pool_shape[1]*pool_shape[2]*pool_shape[3]
    reshaped = tf.reshape(pool,[-1,nodes])
    
    #the first fully connected layer
    fc1_weights = get_weights_variable([nodes,FC_SIZE])
    fc1_bias = tf.Variable(tf.constant(0.1,shape=[FC_SIZE]))
    fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_weights) + fc1_bias)
    fc1 = tf.nn.dropout(fc1,keep_prob)
 
    #the second fully connected layer(output layer)
    fc2_weights = get_weights_variable([FC_SIZE,NUM_CLASSES])
    fc2_bias = tf.Variable(tf.constant(0.1,shape=[NUM_CLASSES]))
    fc2 = tf.nn.relu(tf.matmul(fc1,fc2_weights) + fc2_bias)
    
    return fc2

def evaluation_op(predic_output):
    #calculate TP，TN，FP，FN on test_data
    predictions = tf.argmax(predic_output, 1)
    actuals = tf.argmax(input_ys, 1)

    ones_like_actuals = tf.ones_like(actuals)
    zeros_like_actuals = tf.zeros_like(actuals)
    ones_like_predictions = tf.ones_like(predictions)
    zeros_like_predictions = tf.zeros_like(predictions)

    tn_op = tf.reduce_sum(\
        tf.cast(\
            tf.logical_and(\
            tf.equal(actuals, ones_like_actuals),\
            tf.equal(predictions, ones_like_predictions)\
            ), \
            "float")\
        )

    tp_op = tf.reduce_sum(\
        tf.cast(\
            tf.logical_and(\
            tf.equal(actuals, zeros_like_actuals),\
            tf.equal(predictions, zeros_like_predictions)\
            ),\
            "float")\
        )

    fn_op = tf.reduce_sum(\
        tf.cast(\
          tf.logical_and(\
            tf.equal(actuals, zeros_like_actuals),\
            tf.equal(predictions, ones_like_predictions)\
          ),\
          "float")\
        )

    fp_op = tf.reduce_sum(\
        tf.cast(\
          tf.logical_and(\
            tf.equal(actuals, ones_like_actuals),\
            tf.equal(predictions, zeros_like_predictions)\
          ),\
          "float")\
        )
    return tp_op, tn_op,fp_op, fn_op



def print_test_evaluation(tp,tn,fp,fn):  
    tpr = float(tp)/(float(tp) + float(fn))
    recall = tpr
    print("Sensitivity/recall on the test data is :{}".format(tpr)) 

    specifity = float(tn)/(float(tn) + float(fp))
    print("specifity on the test data is :{}".format(specifity)) 

    precision = float(tp)/(float(tp) + float(fp))
    print("precision on the test data is :{}".format(precision))

    f1_score = (2 * (precision * recall)) / (precision + recall)
    print("f1_score on the test data is :{}".format(f1_score))

    fpr = float(fp)/(float(tp) + float(fn))
    print("fpr on the test data is :{}".format(fpr))

    mcc = ((float(tp) * float(tn)) - (float(fp) * float(fn))) /\
            math.sqrt((float(tp) + float(fp)) * (float(tp) + float(fn))*\
             (float(tn) + float(fp)) * (float(tn) + float(fn)))
    print("mcc on the test data is :{}".format(mcc))

    accuracy = (float(tp) + float(tn))/(float(tp) + float(fp) + float(fn) + float(tn))
    print("accuracy on the test data is :{}".format(accuracy))
    
def cnn_train():   
#     calculate the loss entropy 
    cnn_output = cnn_inference()
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits\
                                (logits=cnn_output,labels=input_ys)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",cross_entropy_mean)
#     optimization
    train_op = tf.train.AdamOptimizer(LR).minimize(cross_entropy_mean)  

#     calculate the accuracy of the model
    correct_prediction = tf.equal(tf.argmax(cnn_output,1), tf.argmax(input_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    tf.summary.scalar("accuracy",accuracy)
#     evaluation on test_data
    tp_op,tn_op,fp_op,fn_op = evaluation_op(cnn_output)

    merged = tf.summary.merge_all()
    saver = tf.train.Saver()
    with tf.Session() as sess:  # run the session        
        writer = tf.summary.FileWriter("logs/filter6", sess.graph)
        sess.run(tf.global_variables_initializer())
        for i in range(TRAINING_ITER): 
            start = (i * BATCH_SIZE)% dataset_size
            end = min(start + BATCH_SIZE,dataset_size)
            batch_xs=training_x_data[start:end]
            batch_ys=training_y_data[start:end]

            _,rs = sess.run([train_op,merged],feed_dict={input_xs:batch_xs,input_ys:batch_ys,\
                                         keep_prob:DROPOUT_KEEP_PROB})
            writer.add_summary(rs,i)
            
#             print loss and accuracy during the training process
            if(i%100==0):
                print("The{}iteration:".format(i))
                print("The cross_entropy_mean为：",end='')
                print(sess.run(cross_entropy_mean,feed_dict={input_xs:batch_xs,\
                                input_ys:batch_ys,keep_prob:DROPOUT_KEEP_PROB}))

                print("The accuracy on training data:",end='')
                print(sess.run(accuracy,feed_dict={input_xs:training_x_data,\
                                input_ys:training_y_data,keep_prob:1}))
                print("The accuracy on test data:",end='')
                print(sess.run(accuracy,feed_dict={input_xs:test_x_data,\
                                           input_ys:test_y_data,keep_prob:1}))
                print("==================")
                
            saver.save(sess,'logs/filter6/model.ckpt')  
                
                
        print("*********training finished********")
        
#         evaluation on test data set

        tp,tn,fp,fn = sess.run([tp_op, tn_op,fp_op, fn_op],\
                feed_dict={input_xs:test_x_data,\
                           input_ys:test_y_data,keep_prob:1})
        print("tp:{},tn:{},fp:{},fn:{}".format(tp,tn,fp,fn))
    
        print_test_evaluation(tp,tn,fp,fn)
        
        
        
def main(argv=None):
    cnn_train()

    
if __name__ == "__main__":
    tf.app.run()





Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

The0iteration:
The cross_entropy_mean为：0.6296295
The accuracy on training data:0.70124483
The accuracy on test data:0.49875313
The100iteration:
The cross_entropy_mean为：0.7373719
The accuracy on training data:0.70124483
The accuracy on test data:0.49875313
The200iteration:
The cross_entropy_mean为：0.24640739
The accuracy on training data:0.9142462
The accuracy on test data:0.86533666
The300iteration:
The cross_entropy_mean为：0.08803768
The accuracy on training data:0.9681881
The accuracy on test data:0.8952618
The400iteration:
The cross_entropy_mean为：0.047690015
The accuracy on training data:0.99723375
The accuracy on test data:0.9077307
The500iteration:
The cross_entropy_mean为：0.048659366
The accuracy on training data:0.99723375
The accuracy on test data:0.90274316
The600iteration:
The cross_entropy

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
